In [1]:
import nltk
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd, numpy as np, re, time
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import classification_report

/home/snig/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [2]:
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [3]:
sarcasm_df = pd.read_csv('sarcasm_df.csv')
sarcasm_df=sarcasm_df.dropna()
#df['polarity'] = df['sentence'].map(lambda text: TextBlob(text).sentiment.polarity)
sarcasm_df.head()

,Unnamed: 0,sentence,is_sarcastic
0,0,former versace store clerk sues over secret 'b...,0.0
1,1,the 'roseanne' revival catches up to our thorn...,0.0
2,2,mom starting to fear son's web series closest ...,1.0
3,3,"boehner just wants wife to listen, not come up...",1.0
4,4,j.k. rowling wishes snape happy birthday in th...,0.0


In [4]:
sarcasm_df = sarcasm_df.sample(frac=1)
# getting features and labels
features = sarcasm_df['sentence']
labels = sarcasm_df['is_sarcastic']

In [5]:
print(type(features))
stopwords = nltk.corpus.stopwords.words('english')
sentence_list_temp=features.tolist()
sentence_list = []

for line in sentence_list_temp:
    temp = []
    for word in line.split():
        if word not in stopwords:
            temp.append(word)
    join = ' '.join([str(ele) for ele in temp])
    sentence_list.append(join)
sarcasm_df['sentence']=sentence_list
sarcasm_df.head()

<class 'pandas.core.series.Series'>


,Unnamed: 0,sentence,is_sarcastic
1271,1271,spring sprung arctic ... way early,0.0
30171,3462,Feb 2012: #Microsoft Slams Google #Privacy Cha...,0.0
81850,64160,I'm happy thunder waited leave happen #sarcasm,1.0
9117,9117,fiona apple's classic 'criminal' video got les...,0.0
90207,75516,Take note people #truestory #true #quote #smil...,1.0


In [9]:
len(sarcasm_df)

100099

In [6]:
# Stemming our data
ps = PorterStemmer()
features=sarcasm_df['sentence']
features= list(map(lambda x: x.split(),features))
features=list(map(lambda x : ''.join([ps.stem(word) for word in x]),features))

In [7]:
# vectorizing the data with maximum of 5000 features
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 5000)
features = list(features)
features = tv.fit_transform(features)


In [8]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = .05, random_state = 0)

NameError: name 'train_test_split' is not defined

In [17]:
#LINEAR SUPPORT VECTOR CLASSIFIER
lsvc = LinearSVC()
lsvc.fit(features_train, labels_train)
predictions = lsvc.predict(features_test)
print(classification_report(labels_test, predictions))
cross_val_score(lsvc, features_train, labels_train, cv=10)

              precision    recall  f1-score   support

         0.0       0.81      0.88      0.84      3556
         1.0       0.61      0.48      0.54      1449

    accuracy                           0.76      5005
   macro avg       0.71      0.68      0.69      5005
weighted avg       0.75      0.76      0.75      5005



array([0.7681388 , 0.75541535, 0.75604627, 0.76456362, 0.76012199,
       0.76706278, 0.75602061, 0.75738774, 0.75770323, 0.76254075])

In [18]:
#LOGISTIC REGRESSION
lr = LogisticRegression()
lr.fit(features_train, labels_train)
predictions = lr.predict(features_test)
print(classification_report(labels_test, predictions))
cross_val_score(lr, features_train, labels_train, cv=10)

/home/snig/snap/jupyter/common/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



              precision    recall  f1-score   support

         0.0       0.80      0.88      0.84      3556
         1.0       0.62      0.46      0.53      1449

    accuracy                           0.76      5005
   macro avg       0.71      0.67      0.68      5005
weighted avg       0.75      0.76      0.75      5005



/home/snig/snap/jupyter/common/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/home/snig/snap/jupyter/common/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/home/snig

array([0.77066246, 0.75573081, 0.75888538, 0.7659306 , 0.76443369,
       0.77011252, 0.7598065 , 0.7598065 , 0.76033232, 0.76338206])

In [24]:
#NAIVE BAYES:
nb = MultinomialNB(alpha=.1)
nb.fit(features_train, labels_train)
predictions = nb.predict(features_test)
print(classification_report(labels_test, predictions))
cross_val_score(nb, features_train, labels_train, cv=10)

              precision    recall  f1-score   support

         0.0       0.80      0.88      0.84      3556
         1.0       0.61      0.46      0.53      1449

    accuracy                           0.76      5005
   macro avg       0.70      0.67      0.68      5005
weighted avg       0.75      0.76      0.75      5005



array([0.76782334, 0.75762355, 0.75867508, 0.76309148, 0.76443369,
       0.76611631, 0.7586497 , 0.75791356, 0.76190977, 0.76516984])

In [29]:
#DECISION TREE
dt = DecisionTreeClassifier(random_state=0)
dt.fit(features_train, labels_train)
predictions = dt.predict(features_test)
print(classification_report(labels_test, predictions))
cross_val_score(dt, features_train, labels_train, cv=10)

              precision    recall  f1-score   support

         0.0       0.82      0.87      0.84      3556
         1.0       0.62      0.52      0.56      1449

    accuracy                           0.77      5005
   macro avg       0.72      0.69      0.70      5005
weighted avg       0.76      0.77      0.76      5005



array([0.77423764, 0.76235542, 0.7617245 , 0.77308097, 0.76790409,
       0.77558103, 0.76338206, 0.76443369, 0.7664318 , 0.77021769])

In [21]:
#RANDOM FOREST
rfc = RandomForestClassifier(n_estimators = 20, random_state = 0)
rfc.fit(features_train, labels_train)
print(classification_report(labels_test, predictions))
cross_val_score(rfc, features_train, labels_train, cv=5)

              precision    recall  f1-score   support

         0.0       0.80      0.88      0.84      3556
         1.0       0.62      0.46      0.53      1449

    accuracy                           0.76      5005
   macro avg       0.71      0.67      0.68      5005
weighted avg       0.75      0.76      0.75      5005



array([0.76775856, 0.76675956, 0.77217519, 0.76586571, 0.77000736])